In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
news_df = pd.read_csv('news_df_cleaned.csv')
news_df

,headline,source,urls,text,classification,first_word,text_cleaned,text_tokenized,text_removed_stopwords,text_lemmatized
0,Tillerson To Visit Africa In First Quarter Of ...,reuters,NaN,U.S. Secretary of State Rex Tillerson is pla...,real,RAMSTEIN,u s secretary of state rex tillerson is plann...,"['u', 's', 'secretary', 'of', 'state', 'rex', ...","['u', 'secretary', 'state', 'rex', 'tillerson'...","['u', 'secretary', 'state', 'rex', 'tillerson'..."
1,Swedish Court Sentences Syrian Asylum Seeker T...,reuters,NaN,A Swedish court on Monday sentenced a Syrian...,real,STOCKHOLM,a swedish court on monday sentenced a syrian ...,"['a', 'swedish', 'court', 'on', 'monday', 'sen...","['swedish', 'court', 'monday', 'sentenced', 's...","['swedish', 'court', 'monday', 'sentenced', 's..."
2,Irish Election Likely After Opposition Submits...,reuters,NaN,An Irish general election appeared likely af...,real,DUBLIN,an irish general election appeared likely aft...,"['an', 'irish', 'general', 'election', 'appear...","['irish', 'general', 'election', 'appeared', '...","['irish', 'general', 'election', 'appeared', '..."
3,Argentina Blocks Some Activists From Attending...,reuters,NaN,Argentina revoked the credentials of some ac...,real,BUENOS,argentina revoked the credentials of some act...,"['argentina', 'revoked', 'the', 'credentials',...","['argentina', 'revoked', 'credentials', 'activ...","['argentina', 'revoked', 'credential', 'activi..."
4,"Kremlin, After Kurdish Vote, Says Supports Int...",reuters,NaN,The Kremlin on Tuesday signaled its oppositi...,real,MOSCOW,the kremlin on tuesday signaled its oppositio...,"['the', 'kremlin', 'on', 'tuesday', 'signaled'...","['kremlin', 'tuesday', 'signaled', 'opposition...","['kremlin', 'tuesday', 'signaled', 'opposition..."
...,...,...,...,...,...,...,...,...,...,...
14757,Obama Said America Wants Him Back For A 3Rd Te...,The Conservative Millennial,ihavethetruth.com,"Earlier this year, in a speech to the Africa...",fake,0,earlier this year in a speech to the african ...,"['earlier', 'this', 'year', 'in', 'a', 'speech...","['earlier', 'year', 'speech', 'african', 'unio...","['earlier', 'year', 'speech', 'african', 'unio..."
14758,Spain To Review Permission For Syria Bound Rus...,Carol Adl,yournewswire.com,"Carol Adl in News , World // 1 Comment Spai...",fake,Carol,carol adl in news world 1 comment spain is ha...,"['carol', 'adl', 'in', 'news', 'world', '1', '...","['carol', 'adl', 'news', 'world', '1', 'commen...","['carol', 'adl', 'news', 'world', '1', 'commen..."
14759,Pakistan Conducts Training Maneuvers At The Bo...,NaN,therussophile.org,This post was originally published on this sit...,fake,This,this post was originally published on this sit...,"['this', 'post', 'was', 'originally', 'publish...","['post', 'originally', 'published', 'site', 'r...","['post', 'originally', 'published', 'site', 'r..."
14760,Us In Danger Of Losing Allies In Asia,Dave Hodges,thecommonsenseshow.com,Philippines President turning his country awa...,fake,Philippines,philippines president turning his country awa...,"['philippines', 'president', 'turning', 'his',...","['philippines', 'president', 'turning', 'count...","['philippine', 'president', 'turning', 'countr..."


TypeError: expected string or bytes-like object